In [20]:
## IMPORTS
import numpy as np
import timeit
start = timeit.default_timer()
from stage import fullPrint
from datetime import datetime, date
from stage import getStage
import pandas as pd
import yahoo_fin.stock_info as yf
import random
from datetime import timedelta
from sklearn.model_selection import train_test_split

In [21]:
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)

def calculateGroupReturn(list):

    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
        except:
            continue
        if not (df['close'] != 0).all():
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn
            
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = calculateGroupReturn(nasdaqList)

In [25]:
indexNum = 0
sum = [1,1]
for i in range(2):
    if i == 0:
        l = train
    else:
        l = test
    index = 0
    while index != len(l):
        if l[index] not in listOfDf.columns:
            l = np.delete(l, index)
        else:
            index += 1
    for index, element in listOfDf[l].iterrows():
        #print(element.to_list())
        listOfStockRet = element.to_list()
        while 1.0 in listOfStockRet:
            listOfStockRet.remove(1.0)
        if len(listOfStockRet) != 0:
            sum[i] = sum[i] * np.mean(listOfStockRet)     
for i in range(2):
    sum[i] = sum[i] / sum[1]
print(sum)

[0.5331015224475909, 1.0]


In [33]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))

def evalOneMax(params):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    print(params)
    winningTrade = 0
    totalTrade = 0
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                if df.at[date,'MA30Slope'] > params[9] and df.loc[date,'RS'] > params[10]:
                    if df.at[date,'close'] > df.at[date,'MA30']:
                        listOfSector.append(df.at[date,'ticker'])
            except: 
                pass
        goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    for symbol in train:
        df = getStage(symbol,params,goodSectorDf)
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        buyPrice = 0
        secondBuyPrice = 0
        drift = -1
        for index, element in df.iterrows():
            if drift == -1:
                drift = index.weekday()
            index = index - timedelta(days= drift)
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                if element.Stage == "Buy" and not inStage:
                    buyPrice = element.close
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    secondBuyPrice = element.close
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                returnRate = element.close / buyPrice
                if returnRate > 1:
                    winningTrade += 1
                    totalTrade += 1
                else:
                    totalTrade += 1
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                    returnRate = element.close / secondBuyPrice
                    if returnRate > 1:
                        winningTrade += 1
                        totalTrade += 1
                    else:
                        totalTrade += 1
                buyPrice = 0
                secondBuyPrice = 0
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    if totalTrade==0:
        print(winningTrade, totalTrade, 0)
    else:
        print(winningTrade, totalTrade, winningTrade/totalTrade)
    return transactionCopy.iloc[-1]['total']

In [34]:
evalOneMax([0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804])

[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804]
55.067328136073705
27388.486947546764
723 1840 0.39293478260869563


27388.486947546764